In [1]:
# IMPORTS
from termite_toolkit import termite 
import pandas as pd
from os import listdir
from pprint import pprint
import xml.etree.ElementTree as ET
import json

# VARIABLES
termite_home = "https://termite.scibite-mvp.nonprod.entellect.com/termite"
sdxml = [f for f in listdir() if f.endswith(".xml")]
core_entities = "ANAT,BIOCHEM,BIOPROC,CELLLINE,CELLTYP,COMPANY,COUNTRY,DBSNP,DRUG,GENE,GOONTOL,INDICATION,MEASURE,MIRNA,MOA,PROTYP,SPECIES"
options = {"format": "node.xml", "output": "json", "entities": core_entities}

In [22]:
ns = {'xocs': 'http://www.elsevier.com/xml/xocs/dtd',
      'ja': 'http://www.elsevier.com/xml/ja/dtd',
     'ce' :'http://www.elsevier.com/xml/common/dtd'}


def xpath2string(value, method='normal'):
        if method == 'normal':
            try:
                valuelist = []
                for v in list(value):
                    valuelist.append(v.text)
                return " : ".join(valuelist)
            except TypeError:
                return ''
        elif method == 'greedy':
            try:
                valuelist = []
                for v in list(value):
                    valuelist.append(ET.tostring(v, method='text').decode("utf-8"))
                    #valuelist.append("".join(map(chr, ET.tostring(v, method='text'))))
                return " ".join(valuelist)
            except AttributeError:
                return ''
        elif method == 'names':
            valuelist = []
            for v in list(value):
                surname_path = v.find('./ce:surname', ns)
                forename_path = v.find('./ce:given-name', ns)
                valuelist.append(dict(surname=surname_path.text, forename=forename_path.text))
            return(valuelist)
        
def tag_using_etree():
    termite_instance = termite.TermiteRequestBuilder()
    termite_instance.set_url(termite_home)
    termite_instance.set_input_format("xml")
    termite_instance.set_entities(core_entities)
    count  = 0
    for fn in sdxml:
        if count > 1:
            break
        count +=1
        metadata = {}
        tree = ET.parse(fn)
        root = tree.getroot()
        jnl_path = tree.findall('.//xocs:srctitle', ns)
        title_path = tree.findall('.//ce:title', ns)
        abstract_path = tree.findall('.//ce:abstract-sec/ce:simple-para', ns)
        affiliation_path = tree.findall('.//ce:affiliation/ce:textfn', ns)
        pubdate_path = tree.findall('.//xocs:meta/xocs:available-online-date', ns)
        author_path = tree.findall('.//ce:author-group/ce:author', ns)
        
        metadata['pii'] = fn[:-4]
        metadata['authors'] = xpath2string(author_path, method='names')
        metadata['published'] = xpath2string(pubdate_path)
        metadata['affiliations'] = xpath2string(affiliation_path)
        metadata['abstract'] = xpath2string(abstract_path, method='greedy')
        metadata['title'] = xpath2string(title_path)
        metadata['journal'] = xpath2string(jnl_path)

        #TODO explore whether we should come in at serial-article and not root
        xmlstr = ET.tostring(root, encoding='utf8', method='xml')
        termite_instance.set_text(xmlstr)
        termite_response = termite_instance.execute()
        df = termite.get_termite_dataframe(termite_response, reject_ambig = False)
        del df['docID']
        del df['realSynList']
        del df['totnosyns']
        del df['nonambigsyns']
        del df['score']
        metadata['entities'] = df.to_dict(orient='records')
        print(json.dumps(metadata, indent=4))
tag_using_etree()

{
    "pii": "S0001209206600331",
    "authors": [
        {
            "surname": "Hallquist",
            "forename": "Deborah L."
        }
    ],
    "published": "2006-08-23",
    "affiliations": "",
    "abstract": "&#8226;DURING TIMES OF WAR, demand for health care providers has stimulated increased employment of nurses and broadened the dimensions of the scope of nursing practice in the surgical arena.&#8226;THIS ARTICLE EXAMINES how the work of army nurses during the Korean War helped develop the role of the RN first assistant. AORN J 82 (October 2005) 644&#8211;647.",
    "title": "Developments in the RN First Assistant Role During the Korean War : Influential Nurses",
    "journal": "AORN Journal",
    "entities": [
        {
            "entityType": "DRUG",
            "hitID": "CHEMBL29",
            "name": "Benzylpenicillin",
            "frag_vector_array": [
                "428#ith open wounds received {!penicillin!} and a blood transfusion.",
                "439#t

In [3]:
def quickway():
    for input_file in sdxml:
        termite_json_response = termite.annotate_files(termite_home, input_file, options)
        display(termite.get_termite_dataframe(termite_json_response, reject_ambig = False))
